[Reference](https://towardsdatascience.com/battle-of-the-data-containers-which-python-typed-structure-is-the-best-6d28fde824e)

In [1]:
import time
from typing import Optional, NoReturn


class Timer:
    _counter_start: Optional[float] = None
    _counter_stop: Optional[float] = None

    def start(self) -> None:
        self._counter_start = time.perf_counter_ns()

    def stop(self) -> None:
        self._counter_stop = time.perf_counter_ns()

    @property
    def time(self) -> float:
        """Time in nano seconds (ns)."""
        self._valid_start_stop()
        return self._counter_stop - self._counter_start  # type: ignore

    def _valid_start_stop(self) -> Optional[NoReturn]:
        if self._counter_start is None:
            raise ValueError("Timer has not been started.")
        if self._counter_stop is None:
            raise ValueError("Timer has not been stopped.")
        return None

In [2]:
# from typing import Protocol
from typing_extensions import Protocol

class ProtoPerson(Protocol):
    name: str
    age: int

In [3]:
pip install Pympler

     |████████████████████████████████| 164 kB 4.0 MB/s 


In [4]:
from typing import Type

from pympler import asizeof # pip install Pympler

def time_person_creation(person_cls: Type[ProtoPerson]) -> float:
    timer = Timer()
    timer.start()
    person_cls(name="Diego", age=33)  # type: ignore
    timer.stop()
    return timer.time


def time_person_get(person_cls: Type[ProtoPerson]) -> float:
    person = person_cls(name="Diego", age=33)  # type: ignore
    timer = Timer()
    timer.start()
    person.age
    timer.stop()
    return timer.time


def time_person_set(person_cls: Type[ProtoPerson]) -> float:
    person = person_cls(name="Diego", age=33)  # type: ignore
    timer = Timer()
    timer.start()
    person.age = 0
    timer.stop()
    return timer.time


def get_person_memory_usage(person_cls: Type[ProtoPerson]) -> int:
    return int(asizeof.asizeof(person_cls(name="Diego", age=33)))  # type: ignore

In [5]:
import statistics
from typing import Type
from dataclasses import dataclass


@dataclass
class PersonTestResult:
    creation_time: float  # ns
    get_time: float  # ns
    set_time: Optional[float]  # ns
    memory: int  # bytes


def test_person_containter(
    person_cls: Type[ProtoPerson], n_trials: int = 100_000, test_set: bool = True
) -> PersonTestResult:
    creation_times = []
    get_times = []
    set_times = []
    for _ in range(n_trials):
        creation_times.append(time_person_creation(person_cls))
        get_times.append(time_person_get(person_cls))
        if test_set:
            set_times.append(time_person_set(person_cls))
    return PersonTestResult(
        creation_time=statistics.mean(creation_times),
        get_time=statistics.mean(get_times),
        set_time=statistics.mean(set_times) if test_set else None,
        memory=get_person_memory_usage(person_cls),
    )

# Regular class

In [6]:
class PersonClass:
    name: str
    age: int

    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age


class PersonClassSlots:
    __slots__ = "name", "age"

    name: str
    age: int

    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

class_test_result = test_person_containter(PersonClass)
class_slots_test_result = test_person_containter(PersonClassSlots)

# Dataclass

In [7]:
from dataclasses import dataclass

@dataclass
class PersonDataClass:
    name: str
    age: int


@dataclass
class PersonDataClassSlots:
    name: str
    age: int

dataclass_test_result = test_person_containter(PersonDataClass)
dataclass_slots_test_result = test_person_containter(PersonDataClassSlots)

# Attrs


In [8]:
 pip install attrs

In [9]:
import attr

@attr.s
class PersonAttrs:
    name: str = attr.ib()
    age: int = attr.ib()

@attr.s(slots=True)
class PersonAttrsSlots:
    name: str = attr.ib()
    age: int = attr.ib()

attrs_test_result = test_person_containter(PersonAttrs)
attrs_slots_test_result = test_person_containter(PersonAttrsSlots)

# Pydantic


In [10]:
pip install pydantic

     |████████████████████████████████| 10.9 MB 3.9 MB/s 


In [11]:
import pydantic

class PydanticPerson(pydantic.BaseModel):
    name: str
    age: int

@pydantic.dataclasses.dataclass
class PydanticPersonDC:
    name: str
    age: int

pydantic_test_result = test_person_containter(PydanticPerson)
pydantic_test_result_dc = test_person_containter(PydanticPersonDC)

# NamedTuple


In [12]:
from typing import NamedTuple

class NTplePerson(NamedTuple):
    name: str
    age: int

named_tuple_test_result = test_person_containter(NTplePerson, test_set=False)

# The control, dict


In [13]:
import statistics

from pympler import asizeof # pip install Pympler

def test_person_dict(n_trials: int = 10_000) -> PersonTestResult:
    timer = Timer()
    creation_times = []
    get_times = []
    set_times = []
    for _ in range(n_trials):
        timer.start()
        person_dict = dict(name="Diego", age=33)
        timer.stop()
        creation_times.append(timer.time)

        timer.start()
        person_dict["age"]
        timer.stop()
        get_times.append(timer.time)

        timer.start()
        person_dict["age"] = 0
        timer.stop()
        set_times.append(timer.time)

    return PersonTestResult(
        creation_time=statistics.mean(creation_times),
        get_time=statistics.mean(get_times),
        set_time=statistics.mean(set_times),
        memory=int(asizeof.asizeof(person_dict)),
    )

dict_test_result = test_person_dict()